In [1]:
import re
import sys
from operator import add

from pyspark.sql import SparkSession

In [2]:
def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)


def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

test, tp =parseNeighbors('1 2')
print test
print tp

1
2


In [3]:
 spark = SparkSession\
        .builder\
        .appName("PythonPageRank")\
        .getOrCreate()

In [4]:
lines = spark.read.text('data/pagerank_data.txt').rdd.map(lambda r: r[0])

In [5]:
lines.take(2)

[u'1 2', u'1 3']

In [6]:
lines.take(2)

[u'1 2', u'1 3']

In [7]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [8]:
pg = sc.textFile('data/pagerank_data.txt')

In [9]:
pg.take(1)

[u'1 2']

In [12]:
# links = lines.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()
links = pg.map(lambda urls: parseNeighbors(urls))
# pg.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().mapValues(list).collect()

In [13]:
links.collect()

[(u'1', u'2'),
 (u'1', u'3'),
 (u'1', u'4'),
 (u'2', u'1'),
 (u'3', u'1'),
 (u'4', u'1')]

In [14]:
links.take(1)

[(u'1', u'2')]

In [15]:
ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))


In [16]:
ranks.collect()

[(u'1', 1.0), (u'1', 1.0), (u'1', 1.0), (u'2', 1.0), (u'3', 1.0), (u'4', 1.0)]

In [17]:
for iteration in range(2):
    # Calculates URL contributions to the rank of other URLs.
    contribs = links.join(ranks).flatMap(
        lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))

    # Re-calculates URL ranks based on neighbor contributions.
    ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)
ranks.take(2)

[(u'2', 2.445), (u'4', 2.445)]

In [18]:
for (link, rank) in ranks.collect():
    print("%s has rank: %s." % (link, rank))
    

2 has rank: 2.445.
4 has rank: 2.445.
1 has rank: 7.035.
3 has rank: 2.445.


In [ ]:
import pandas as pd
